In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Sampler
from PIL import Image, ImageOps
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from resnest.torch import resnest50



import time
import pickle
import numpy as np
from torchvision.transforms import Lambda
import argparse
import copy
import random
import numbers
from torch.utils.tensorboard import SummaryWriter
from sklearn import metrics
import os

In [2]:
from NLBlock_MutiConv import NLBlock
from NLBlock_MutiConv import TimeConv

In [3]:
if torch.cuda.is_available()==True:
    device="cuda:2"
else:
    device ="cpu"
    
print(torch.cuda.is_available())

True


In [3]:
class M2_densenet_lstm(torch.nn.Module):
    def __init__(self):
        super(M2_densenet_lstm, self).__init__()
        resnet = models.densenet169(pretrained=True) #pretrained=True
        self.share = torch.nn.Sequential()
        self.share.add_module("features", resnet.features)
        #self.share.add_module("avgpool", resnet.avgpool)
        self.avg = nn.AvgPool2d(6)

        # self.share.add_module("classifier", resnet.classifier)
        
        #self.fc_1 = nn.Linear(9216, 4096)
        
        # 
        self.lstm = nn.LSTM(1664, 512, batch_first=True)
#        self.lstm = nn.LSTM(2028, 512, batch_first=True)
        self.fc = nn.Linear(512, 19)

        self.dropout = nn.Dropout(p=0.2)

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = x.view(-1, 3, 216, 216)
        x = self.share.forward(x) # ([100, 1664, 6, 6])   # ([100,2048,1,1])
        x = self.avg(x)
        x = x.view(-1, sequence_length, 1664)  
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = self.dropout(y)
        y = self.fc(y)
        return y

In [7]:
M2 = M2_densenet_lstm()
#LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_densenet_M2.pth"
LFB_path = "/media/yilin/tmrnet/models/temp/densenet169/sl10_lr1e-5_6-25train/latest_model_18.pth"

In [8]:
M2.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [23]:
M2.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [4]:
class M5_densenet_lstm_nl(torch.nn.Module):
    def __init__(self):
        super(M5_densenet_lstm_nl, self).__init__()
        densenet = models.densenet169(pretrained=True) #pretrained=True
        self.share = torch.nn.Sequential()
        self.share.add_module("features", densenet.features)
        #self.share.add_module("avgpool", resnet.avgpool)
        self.avg = nn.AvgPool2d(6)

        # self.share.add_module("classifier", resnet.classifier)
        
        #self.fc_1 = nn.Linear(9216, 4096)
         
        self.lstm = nn.LSTM(1664, 512, batch_first=True)
#        self.lstm = nn.LSTM(2028, 512, batch_first=True)
        #self.fc = nn.Linear(512, 19)
        self.fc_c = nn.Linear(512, 19) #7
        self.fc_h_c = nn.Linear(1024, 512)
        self.nl_block = NLBlock()
        self.dropout = nn.Dropout(p=0.5)

        
        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc_c.weight)
        init.xavier_uniform_(self.fc_h_c.weight)

    def forward(self, x,long_feature):
        x = x.view(-1, 3, 216, 216)
        x = self.share.forward(x) # ([100, 1664, 6, 6])   # ([100,2048,1,1])
        x = self.avg(x)
        x = x.view(-1, sequence_length, 1664)  
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        
        
        y_1 = self.nl_block(y, long_feature)
        y = torch.cat([y, y_1], dim=1)
        y = self.dropout(self.fc_h_c(y))
        y = F.relu(y)
        y = self.fc_c(y)
        return y

In [9]:
M5 = M5_densenet_lstm_nl()
LFB_path = "/media/yilin/pre-trained models/CataRCNet_models/densenet_non-local-only_M5.pth"

In [10]:
M5.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [20]:
class M8_densenet_lstm_nl_tvl(torch.nn.Module):
    def __init__(self):
        super(M8_densenet_lstm_nl_tvl, self).__init__()
        densenet = models.densenet169(pretrained=True) #pretrained=True
        self.share = torch.nn.Sequential()
        self.share.add_module("features", densenet.features)
        #self.share.add_module("avgpool", resnet.avgpool)
        self.avg = nn.AvgPool2d(6)

        # self.share.add_module("classifier", resnet.classifier)
        
        #self.fc_1 = nn.Linear(9216, 4096)
        
        # 
        self.lstm = nn.LSTM(1664, 512, batch_first=True)
#        self.lstm = nn.LSTM(2028, 512, batch_first=True)
        #self.fc = nn.Linear(512, 19)
        self.fc_c = nn.Linear(512, 19) #7
        self.fc_h_c = nn.Linear(1024, 512)
        self.nl_block = NLBlock()
        self.dropout = nn.Dropout(p=0.5)
        
        self.time_conv = TimeConv()

        

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc_c.weight)
        init.xavier_uniform_(self.fc_h_c.weight)

    def forward(self, x, long_feature):
        x = x.view(-1, 3, 216, 216)
        x = self.share.forward(x) # ([100, 1664, 6, 6])   # ([100,2048,1,1])
        x = self.avg(x)
        x = x.view(-1, sequence_length, 1664)  
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        
        Lt = self.time_conv(long_feature)

        y_1 = self.nl_block(y, Lt)
        y = torch.cat([y, y_1], dim=1)
        y = self.dropout(self.fc_h_c(y))
        y = F.relu(y)
        y = self.fc_c(y)
        return y

In [26]:
M8 = M8_densenet_lstm_nl_tvl()
LFB_path = "/media/yilin/tmrnet/models/temp/non-local/densenet_pretrained_lr1e-5_L30_multiconv_2fc_6-25train/latest_model_6.pth"



In [27]:
M8.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [5]:
class LFB_densenet_lstm(torch.nn.Module):
    def __init__(self):
        super(LFB_densenet_lstm, self).__init__()
        densenet = models.densenet169(pretrained=True) #pretrained=True
        self.share = torch.nn.Sequential()
        self.share.add_module("features", densenet.features)
        #self.share.add_module("avgpool", resnet.avgpool)
        self.avg = nn.AvgPool2d(6)

        # self.share.add_module("classifier", resnet.classifier)
        
        #self.fc_1 = nn.Linear(9216, 4096)
        
        # 
        self.lstm = nn.LSTM(1664, 512, batch_first=True)
#        self.lstm = nn.LSTM(2028, 512, batch_first=True)
        #self.fc = nn.Linear(512, 19)

        

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        #init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = x.view(-1, 3, 216, 216)
        x = self.share.forward(x) # ([100, 1664, 6, 6])   # ([100,2048,1,1])
        x = self.avg(x)
        x = x.view(-1, sequence_length, 1664)  
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        return y

# LOSS FUNCTION


In [6]:
class WeightedCrossEntropy(torch.nn.Module):
    '''
    WCE
    '''       
     # 6-25
    def __init__(self, weight=torch.Tensor([0.0033, 0.4182, 0.1321, 0.0234, 0.0344, 0.0146, 0.0428, 0.0140, 0.0092,
        0.0272, 0.0096, 0.0323, 0.0341, 0.0508, 0.0151, 0.0160, 0.0365, 0.0738,
        0.0128])):
        super(WeightedCrossEntropy, self).__init__()
        
        weight = weight.to(device)
        self.weighted_cross_entropy = nn.CrossEntropyLoss(weight=weight)
        
    def forward(self, inputs, target):
        return self.weighted_cross_entropy.forward(inputs, target)

In [7]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

class CataractsDataset(Dataset):
    def __init__(self, file_paths,file_labels, transform=None,loader=pil_loader):
        self.file_paths = file_paths
        self.file_labels_phase = file_labels[:,0]
        self.transform = transform
        self.loader = loader
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        img_names = self.file_paths[index]
        labels = self.file_labels_phase[index]
        imgs = self.loader(img_names)
        if self.transform is not None:
            imgs = self.transform(imgs)

        return imgs, labels, index

In [8]:
######## center
def get_test_data(data_path):
    with open(data_path, 'rb') as f:
        test_paths_labels = pickle.load(f)

    test_paths = test_paths_labels[0]
    test_labels = test_paths_labels[1]
    test_num_each = test_paths_labels[2]

    print('test_paths   : {:6d}'.format(len(test_paths)))
    print('test_labels  : {:6d}'.format(len(test_labels)))

    test_labels = np.asarray(test_labels, dtype=np.int64)

    test_transforms = None
################################### 
    test_transforms = transforms.Compose([
            transforms.CenterCrop(216),  #transforms.RandomCrop(216),
            transforms.ToTensor()
        ])


    test_dataset = CataractsDataset(test_paths, test_labels, test_transforms)

    return test_dataset, test_num_each

# Test for M2

In [9]:
class SequenceSampler(Sampler):
    def __init__(self, data_source, idx):
        super().__init__(data_source)
        self.data_source = data_source
        self.idx = idx

    def __iter__(self):
        return iter(self.idx)

    def __len__(self):
        return len(self.idx)

In [10]:
# for sliding window
def get_start_idx(sequence_length, list_each_length):
    count = 0
    idx = []
    for i in range(len(list_each_length)):
        for j in range(count, count + (list_each_length[i] + 1 - sequence_length)):
            idx.append(j)
        count += list_each_length[i]
    return idx

In [11]:
def test_model(model,test_dataset, test_num_each):
    num_test = len(test_dataset)
    test_useful_start_idx = get_start_idx(sequence_length, test_num_each)
    test_useful_start_idx_LFB = get_start_idx(sequence_length, test_num_each)

    num_test_we_use = len(test_useful_start_idx)
    num_test_we_use_LFB = len(test_useful_start_idx_LFB)

    test_we_use_start_idx = test_useful_start_idx[0:num_test_we_use]
    test_we_use_start_idx_LFB = test_useful_start_idx_LFB[0:num_test_we_use_LFB]

    test_idx = []
    for i in range(num_test_we_use):
        for j in range(sequence_length):
            test_idx.append(test_we_use_start_idx[i] + j)

    num_test_all = len(test_idx)

    test_idx_LFB = []
    for i in range(num_test_we_use_LFB):
        for j in range(sequence_length):
            test_idx_LFB.append(test_we_use_start_idx_LFB[i] + j)

    dict_index, dict_value = zip(*list(enumerate(test_we_use_start_idx_LFB)))
    dict_test_start_idx_LFB = dict(zip(dict_value, dict_index))

    print('num test start idx : {:6d}'.format(len(test_useful_start_idx)))
    print('last idx test start: {:6d}'.format(test_useful_start_idx[-1]))
    print('num of test dataset: {:6d}'.format(num_test))
    print('num of test we use : {:6d}'.format(num_test_we_use))
    print('num of all test use: {:6d}'.format(num_test_all))
   # TODO sampler

    test_loader = DataLoader(test_dataset,
                             batch_size=test_batch_size,
                             sampler=SequenceSampler(test_dataset, test_idx),
                             num_workers=workers)

    
    torch.cuda.empty_cache()
    
    ###########################3
    # model = resnet_lstm()
    #model.load_state_dict(torch.load(model_path))
    model.load_state_dict(torch.load(LFB_path))
    
    ###############################
    

    model.to(device)

    criterion = WeightedCrossEntropy()   #size_average=False

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    test_start_time = time.time()

    all_preds = []
    all_preds_score = []


    with torch.no_grad():

        for data in test_loader:
            

            torch.cuda.empty_cache()            


            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels[(sequence_length - 1)::sequence_length]


            inputs = inputs.view(-1, sequence_length, 3, 216,216)


            outputs = model.forward(inputs)

            outputs = outputs[sequence_length - 1::sequence_length]
            # softmax
            Sm = nn.Softmax()
            outputs = Sm(outputs)
            possibility, preds = torch.max(outputs.data, 1)


            for i in range(len(preds)):
                all_preds.append(preds[i].data.cpu())
            for i in range(len(possibility)):
                all_preds_score.append(possibility[i].data.cpu())
            
            
            loss = criterion(outputs, labels)


            test_loss += loss.data.item()
            test_corrects += torch.sum(preds == labels.data)
            print("test_corrects:",test_corrects)

    test_elapsed_time = time.time() - test_start_time
    test_accuracy = float(test_corrects) / float(num_test_we_use)
    test_average_loss = test_loss / num_test_we_use

    print('type of all_preds:', type(all_preds))
    print('leng of all preds:', len(all_preds))
    save_test = int("{:4.0f}".format(test_accuracy * 10000))
    pred_name = "./test_results_pkl/"+model_pure_name + '_test_' + str(save_test)  + '.pkl'
    

    with open(pred_name, 'wb') as f:
        pickle.dump(all_preds, f)

    print('test elapsed: {:2.0f}m{:2.0f}s'
          ' test loss: {:4.4f}'
          ' test accu: {:.4f}'
          .format(test_elapsed_time // 60,
                  test_elapsed_time % 60,
                  test_average_loss, test_accuracy))

In [37]:
# TEST25
test_dataset, test_num_each = get_test_data('../gen_datasets/test_paths_labels.pkl')

test_paths   :  15425
test_labels  :  15425


In [38]:
# example output of 2 test videos, 
# if you cannot load the pretrained model, you can change the layer name self.cnn to self.share

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2   
MODEL = M2_densenet_lstm() # M2_densenet_lstm() /  M3_resnest_lstm()


#########################################
device = "cuda:2"  

model_pure_name= 'LFB-densenet'
 # 'resnet/sl10_lr5e-5_6-25train/' cuda:0
LFB_path = "/media/yilin/tmrnet/models/temp/densenet169/sl10_lr5e-6_6-25train/latest_model_5.pth"


# test M2
test_model(MODEL, test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000


/tmp/ipykernel_33777/935845069.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:2')
test_corrects: tensor(15, device='cuda:2')
test_corrects: tensor(15, device='cuda:2')
test_corrects: tensor(17, device='cuda:2')
test_corrects: tensor(23, device='cuda:2')
test_corrects: tensor(23, device='cuda:2')
test_corrects: tensor(32, device='cuda:2')
test_corrects: tensor(42, device='cuda:2')
test_corrects: tensor(51, device='cuda:2')
test_corrects: tensor(54, device='cuda:2')
test_corrects: tensor(55, device='cuda:2')
test_corrects: tensor(61, device='cuda:2')
test_corrects: tensor(70, device='cuda:2')
test_corrects: tensor(80, device='cuda:2')
test_corrects: tensor(90, device='cuda:2')
test_corrects: tensor(99, device='cuda:2')
test_corrects: tensor(99, device='cuda:2')
test_corrects: tensor(99, device='cuda:2')
test_corrects: tensor(99, device='cuda:2')
test_corrects: tensor(105, device='cuda:2')
test_corrects: tensor(114, device='cuda:2')
test_corrects: tensor(120, device='cuda:2')
test_corrects: tensor(122, device='cuda:2')
test_co

test_corrects: tensor(888, device='cuda:2')
test_corrects: tensor(890, device='cuda:2')
test_corrects: tensor(891, device='cuda:2')
test_corrects: tensor(891, device='cuda:2')
test_corrects: tensor(897, device='cuda:2')
test_corrects: tensor(907, device='cuda:2')
test_corrects: tensor(916, device='cuda:2')
test_corrects: tensor(921, device='cuda:2')
test_corrects: tensor(923, device='cuda:2')
test_corrects: tensor(925, device='cuda:2')
test_corrects: tensor(932, device='cuda:2')
test_corrects: tensor(932, device='cuda:2')
test_corrects: tensor(940, device='cuda:2')
test_corrects: tensor(950, device='cuda:2')
test_corrects: tensor(960, device='cuda:2')
test_corrects: tensor(970, device='cuda:2')
test_corrects: tensor(980, device='cuda:2')
test_corrects: tensor(989, device='cuda:2')
test_corrects: tensor(996, device='cuda:2')
test_corrects: tensor(1006, device='cuda:2')
test_corrects: tensor(1016, device='cuda:2')
test_corrects: tensor(1026, device='cuda:2')
test_corrects: tensor(1031, d

test_corrects: tensor(2190, device='cuda:2')
test_corrects: tensor(2200, device='cuda:2')
test_corrects: tensor(2210, device='cuda:2')
test_corrects: tensor(2220, device='cuda:2')
test_corrects: tensor(2228, device='cuda:2')
test_corrects: tensor(2238, device='cuda:2')
test_corrects: tensor(2248, device='cuda:2')
test_corrects: tensor(2258, device='cuda:2')
test_corrects: tensor(2268, device='cuda:2')
test_corrects: tensor(2278, device='cuda:2')
test_corrects: tensor(2286, device='cuda:2')
test_corrects: tensor(2296, device='cuda:2')
test_corrects: tensor(2306, device='cuda:2')
test_corrects: tensor(2316, device='cuda:2')
test_corrects: tensor(2326, device='cuda:2')
test_corrects: tensor(2335, device='cuda:2')
test_corrects: tensor(2341, device='cuda:2')
test_corrects: tensor(2346, device='cuda:2')
test_corrects: tensor(2355, device='cuda:2')
test_corrects: tensor(2364, device='cuda:2')
test_corrects: tensor(2373, device='cuda:2')
test_corrects: tensor(2383, device='cuda:2')
test_corre

...

test_corrects: tensor(4812, device='cuda:2')
test_corrects: tensor(4822, device='cuda:2')
test_corrects: tensor(4827, device='cuda:2')
test_corrects: tensor(4829, device='cuda:2')
test_corrects: tensor(4832, device='cuda:2')
test_corrects: tensor(4839, device='cuda:2')
test_corrects: tensor(4848, device='cuda:2')
test_corrects: tensor(4854, device='cuda:2')
test_corrects: tensor(4863, device='cuda:2')
test_corrects: tensor(4873, device='cuda:2')
test_corrects: tensor(4883, device='cuda:2')
test_corrects: tensor(4893, device='cuda:2')
test_corrects: tensor(4903, device='cuda:2')
test_corrects: tensor(4913, device='cuda:2')
test_corrects: tensor(4923, device='cuda:2')
test_corrects: tensor(4933, device='cuda:2')
test_corrects: tensor(4943, device='cuda:2')
test_corrects: tensor(4946, device='cuda:2')
test_corrects: tensor(4952, device='cuda:2')
test_corrects: tensor(4962, device='cuda:2')
test_corrects: tensor(4972, device='cuda:2')
test_corrects: tensor(4982, device='cuda:2')
test_corre

test_corrects: tensor(6220, device='cuda:2')
test_corrects: tensor(6230, device='cuda:2')
test_corrects: tensor(6240, device='cuda:2')
test_corrects: tensor(6250, device='cuda:2')
test_corrects: tensor(6260, device='cuda:2')
test_corrects: tensor(6270, device='cuda:2')
test_corrects: tensor(6279, device='cuda:2')
test_corrects: tensor(6279, device='cuda:2')
test_corrects: tensor(6287, device='cuda:2')
test_corrects: tensor(6296, device='cuda:2')
test_corrects: tensor(6306, device='cuda:2')
test_corrects: tensor(6316, device='cuda:2')
test_corrects: tensor(6326, device='cuda:2')
test_corrects: tensor(6336, device='cuda:2')
test_corrects: tensor(6346, device='cuda:2')
test_corrects: tensor(6356, device='cuda:2')
test_corrects: tensor(6366, device='cuda:2')
test_corrects: tensor(6376, device='cuda:2')
test_corrects: tensor(6385, device='cuda:2')
test_corrects: tensor(6395, device='cuda:2')
test_corrects: tensor(6405, device='cuda:2')
test_corrects: tensor(6414, device='cuda:2')
test_corre

test_corrects: tensor(7714, device='cuda:2')
test_corrects: tensor(7717, device='cuda:2')
test_corrects: tensor(7722, device='cuda:2')
test_corrects: tensor(7723, device='cuda:2')
test_corrects: tensor(7723, device='cuda:2')
test_corrects: tensor(7727, device='cuda:2')
test_corrects: tensor(7737, device='cuda:2')
test_corrects: tensor(7747, device='cuda:2')
test_corrects: tensor(7757, device='cuda:2')
test_corrects: tensor(7767, device='cuda:2')
test_corrects: tensor(7774, device='cuda:2')
test_corrects: tensor(7783, device='cuda:2')
test_corrects: tensor(7793, device='cuda:2')
test_corrects: tensor(7796, device='cuda:2')
test_corrects: tensor(7799, device='cuda:2')
test_corrects: tensor(7809, device='cuda:2')
test_corrects: tensor(7819, device='cuda:2')
test_corrects: tensor(7827, device='cuda:2')
test_corrects: tensor(7836, device='cuda:2')
test_corrects: tensor(7839, device='cuda:2')
test_corrects: tensor(7849, device='cuda:2')
test_corrects: tensor(7859, device='cuda:2')
test_corre

test_corrects: tensor(8879, device='cuda:2')
test_corrects: tensor(8889, device='cuda:2')
test_corrects: tensor(8895, device='cuda:2')
test_corrects: tensor(8904, device='cuda:2')
test_corrects: tensor(8914, device='cuda:2')
test_corrects: tensor(8924, device='cuda:2')
test_corrects: tensor(8934, device='cuda:2')
test_corrects: tensor(8944, device='cuda:2')
test_corrects: tensor(8952, device='cuda:2')
test_corrects: tensor(8958, device='cuda:2')
test_corrects: tensor(8963, device='cuda:2')
test_corrects: tensor(8967, device='cuda:2')
test_corrects: tensor(8975, device='cuda:2')
test_corrects: tensor(8975, device='cuda:2')
test_corrects: tensor(8978, device='cuda:2')
test_corrects: tensor(8985, device='cuda:2')
test_corrects: tensor(8993, device='cuda:2')
test_corrects: tensor(9003, device='cuda:2')
test_corrects: tensor(9013, device='cuda:2')
test_corrects: tensor(9023, device='cuda:2')
test_corrects: tensor(9033, device='cuda:2')
test_corrects: tensor(9041, device='cuda:2')
test_corre

test_corrects: tensor(10319, device='cuda:2')
test_corrects: tensor(10320, device='cuda:2')
test_corrects: tensor(10328, device='cuda:2')
test_corrects: tensor(10332, device='cuda:2')
test_corrects: tensor(10332, device='cuda:2')
test_corrects: tensor(10332, device='cuda:2')
test_corrects: tensor(10334, device='cuda:2')
test_corrects: tensor(10344, device='cuda:2')
test_corrects: tensor(10354, device='cuda:2')
test_corrects: tensor(10364, device='cuda:2')
test_corrects: tensor(10374, device='cuda:2')
test_corrects: tensor(10384, device='cuda:2')
test_corrects: tensor(10391, device='cuda:2')
test_corrects: tensor(10392, device='cuda:2')
test_corrects: tensor(10397, device='cuda:2')
test_corrects: tensor(10407, device='cuda:2')
test_corrects: tensor(10415, device='cuda:2')
test_corrects: tensor(10419, device='cuda:2')
test_corrects: tensor(10429, device='cuda:2')
test_corrects: tensor(10439, device='cuda:2')
test_corrects: tensor(10449, device='cuda:2')
test_corrects: tensor(10459, devic

# TESTM5 M8

In [12]:
# Long Term Feature bank
g_LFB_train = np.zeros(shape=(0, 512))
g_LFB_val = np.zeros(shape=(0, 512))

In [13]:
def get_long_feature(start_index_list, dict_start_idx_LFB, lfb):
    long_feature = []
    for j in range(len(start_index_list)):
        long_feature_each = []
        
        # 上一个存在feature的index
        last_LFB_index_no_empty = dict_start_idx_LFB[int(start_index_list[j])]
        
        # 
        for k in range(LFB_length):
            LFB_index = (start_index_list[j] - k - 1)
            if int(LFB_index) in dict_start_idx_LFB:                
                LFB_index = dict_start_idx_LFB[int(LFB_index)]
                long_feature_each.append(lfb[LFB_index])
                last_LFB_index_no_empty = LFB_index
            else:
                long_feature_each.append(lfb[last_LFB_index_no_empty])
            
        long_feature.append(long_feature_each)
    return long_feature

In [14]:
### m4 m7
g_LFB_test = np.zeros(shape=(0, 512))

def test_model(model,test_dataset, test_num_each):
    num_test = len(test_dataset)
    test_useful_start_idx = get_start_idx(sequence_length, test_num_each)
    test_useful_start_idx_LFB = get_start_idx(sequence_length, test_num_each)

    num_test_we_use = len(test_useful_start_idx)
    num_test_we_use_LFB = len(test_useful_start_idx_LFB)

    test_we_use_start_idx = test_useful_start_idx[0:num_test_we_use]
    test_we_use_start_idx_LFB = test_useful_start_idx_LFB[0:num_test_we_use_LFB]

    test_idx = []
    for i in range(num_test_we_use):
        for j in range(sequence_length):
            test_idx.append(test_we_use_start_idx[i] + j)

    num_test_all = len(test_idx)

    test_idx_LFB = []
    for i in range(num_test_we_use_LFB):
        for j in range(sequence_length):
            test_idx_LFB.append(test_we_use_start_idx_LFB[i] + j)

    dict_index, dict_value = zip(*list(enumerate(test_we_use_start_idx_LFB)))
    dict_test_start_idx_LFB = dict(zip(dict_value, dict_index))

    print('num test start idx : {:6d}'.format(len(test_useful_start_idx)))
    print('last idx test start: {:6d}'.format(test_useful_start_idx[-1]))
    print('num of test dataset: {:6d}'.format(num_test))
    print('num of test we use : {:6d}'.format(num_test_we_use))
    print('num of all test use: {:6d}'.format(num_test_all))
# TODO sampler

    test_loader = DataLoader(test_dataset,
                             batch_size=test_batch_size,
                             sampler=SequenceSampler(test_dataset, test_idx),
                             num_workers=workers)

    global g_LFB_test
    print("loading features!>.........")

    if not load_exist_LFB:
        g_LFB_test = np.zeros(shape=(0, 512))

        test_feature_loader = DataLoader(
            test_dataset,
            batch_size=test_batch_size,
            sampler=SequenceSampler(test_dataset, test_idx_LFB),
            num_workers=workers,
            pin_memory=False
        )

        ####################
        # model_LFB = resnet_lstm_LFB()

        model_LFB.load_state_dict(torch.load(LFB_path), strict=False)        
        ########################

        model_LFB.to(device)

        for params in model_LFB.parameters():
            params.requires_grad = False

        model_LFB.eval()

        with torch.no_grad():

            for data in test_feature_loader:

                inputs, labels_phase = data[0].to(device), data[1].to(device)
                

                inputs = inputs.view(-1, sequence_length, 3, 216,216)  #
                outputs_feature = model_LFB.forward(inputs)

                for j in range(len(outputs_feature)):
                    save_feature = outputs_feature.data.cpu()[j].numpy()
                    save_feature = save_feature.reshape(1, 512)
                    g_LFB_test = np.concatenate((g_LFB_test, save_feature), axis=0)

                print("test feature length:", len(g_LFB_test))

        print("finish!")
        g_LFB_test = np.array(g_LFB_test)
        
#         LFB_train_path_save_path = "./LFB/LFB_test_resnet.pkl"

        with open(LFB_test_path_save_path, 'wb') as f: 
            #"../Training TMRNet/LFB/g_LFB_test.pkl"
            # with open("./LFB/g_LFB_test.pkl", 'wb') as f:
            pickle.dump(g_LFB_test, f)

    else:
        with open(LFB_test_path_save_path, 'rb') as f: 
            g_LFB_test = pickle.load(f)

        print("load completed")

    print("g_LFB_test shape:", g_LFB_test.shape)    
    
        
    
    torch.cuda.empty_cache()
    
    ###########################3
    # model = resnet_lstm()
    #model.load_state_dict(torch.load(model_path))
    model.load_state_dict(torch.load(model_path))
    
    ###############################
    

    model.to(device)

    criterion = WeightedCrossEntropy()   #size_average=False

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    test_start_time = time.time()

    all_preds = []
    all_preds_score = []


    with torch.no_grad():

        for data in test_loader:
            
            torch.cuda.empty_cache()            


            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels[(sequence_length - 1)::sequence_length]


            start_index_list = data[2]
            start_index_list = start_index_list[0::sequence_length]
            long_feature = get_long_feature(start_index_list=start_index_list,
                                            dict_start_idx_LFB=dict_test_start_idx_LFB,
                                            lfb=g_LFB_test)
            long_feature = torch.Tensor(long_feature).to(device)

            inputs = inputs.view(-1, sequence_length, 3, 216,216)

            outputs = model.forward(inputs, long_feature=long_feature)

            # softmax
            Sm = nn.Softmax()
            outputs = Sm(outputs)
            possibility, preds = torch.max(outputs.data, 1)


            for i in range(len(preds)):
                all_preds.append(preds[i].data.cpu())
            for i in range(len(possibility)):
                all_preds_score.append(possibility[i].data.cpu())
            
            
            loss = criterion(outputs, labels)


            test_loss += loss.data.item()
            test_corrects += torch.sum(preds == labels.data)
            print("test_corrects:",test_corrects)

    test_elapsed_time = time.time() - test_start_time
    test_accuracy = float(test_corrects) / float(num_test_we_use)
    test_average_loss = test_loss / num_test_we_use

    print('type of all_preds:', type(all_preds))
    print('leng of all preds:', len(all_preds))
    save_test = int("{:4.0f}".format(test_accuracy * 10000))
    pred_name = "./test_results_pkl/"+model_pure_name + '_test_' + str(save_test)  + '.pkl'
    

    with open(pred_name, 'wb') as f:
        pickle.dump(all_preds, f)

    print('test elapsed: {:2.0f}m{:2.0f}s'
          ' test loss: {:4.4f}'
          ' test accu: {:.4f}'
          .format(test_elapsed_time // 60,
                  test_elapsed_time % 60,
                  test_average_loss, test_accuracy))



In [15]:
test_dataset, test_num_each = get_test_data('../gen_datasets/test_paths_labels.pkl')

test_paths   :  15425
test_labels  :  15425


In [16]:
len(test_num_each)

25

In [20]:
# the first time to test, it is necessary to load feature bank

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = False # False

MODEL = M5_densenet_lstm_nl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_densenet_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_densenet.pkl"




#########################################
device = "cuda:1"  
model_pure_name= 'NL-densenet' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/pre-trained models/CataRCNet_models/densenet_non-local-only_M5.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/densenet169/sl10_lr5e-6_6-25train/latest_model_5.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
test feature length: 10
test feature length: 20
test feature length: 30
test feature length: 40
test feature length: 50
test feature length: 60
test feature length: 70
test feature length: 80
test feature length: 90
test feature length: 100
test feature length: 110
test feature length: 120
test feature length: 130
test feature length: 140
test feature length: 150
test feature length: 160
test feature length: 170
test feature length: 180
test feature length: 190
test feature length: 200
test feature length: 210
test feature length: 220
test feature length: 230
test feature length: 240
test feature length: 250
test feature length: 260
test feature length: 270
test feature length: 280
test feature length: 290
test feature length: 300
test feature length: 310
test feature length: 320
test feature length: 330
test feature len

test feature length: 3150
test feature length: 3160
test feature length: 3170
test feature length: 3180
test feature length: 3190
test feature length: 3200
test feature length: 3210
test feature length: 3220
test feature length: 3230
test feature length: 3240
test feature length: 3250
test feature length: 3260
test feature length: 3270
test feature length: 3280
test feature length: 3290
test feature length: 3300
test feature length: 3310
test feature length: 3320
test feature length: 3330
test feature length: 3340
test feature length: 3350
test feature length: 3360
test feature length: 3370
test feature length: 3380
test feature length: 3390
test feature length: 3400
test feature length: 3410
test feature length: 3420
test feature length: 3430
test feature length: 3440
test feature length: 3450
test feature length: 3460
test feature length: 3470
test feature length: 3480
test feature length: 3490
test feature length: 3500
test feature length: 3510
test feature length: 3520
test feature

test feature length: 6310
test feature length: 6320
test feature length: 6330
test feature length: 6340
test feature length: 6350
test feature length: 6360
test feature length: 6370
test feature length: 6380
test feature length: 6390
test feature length: 6400
test feature length: 6410
test feature length: 6420
test feature length: 6430
test feature length: 6440
test feature length: 6450
test feature length: 6460
test feature length: 6470
test feature length: 6480
test feature length: 6490
test feature length: 6500
test feature length: 6510
test feature length: 6520
test feature length: 6530
test feature length: 6540
test feature length: 6550
test feature length: 6560
test feature length: 6570
test feature length: 6580
test feature length: 6590
test feature length: 6600
test feature length: 6610
test feature length: 6620
test feature length: 6630
test feature length: 6640
test feature length: 6650
test feature length: 6660
test feature length: 6670
test feature length: 6680
test feature

test feature length: 9470
test feature length: 9480
test feature length: 9490
test feature length: 9500
test feature length: 9510
test feature length: 9520
test feature length: 9530
test feature length: 9540
test feature length: 9550
test feature length: 9560
test feature length: 9570
test feature length: 9580
test feature length: 9590
test feature length: 9600
test feature length: 9610
test feature length: 9620
test feature length: 9630
test feature length: 9640
test feature length: 9650
test feature length: 9660
test feature length: 9670
test feature length: 9680
test feature length: 9690
test feature length: 9700
test feature length: 9710
test feature length: 9720
test feature length: 9730
test feature length: 9740
test feature length: 9750
test feature length: 9760
test feature length: 9770
test feature length: 9780
test feature length: 9790
test feature length: 9800
test feature length: 9810
test feature length: 9820
test feature length: 9830
test feature length: 9840
test feature

test feature length: 12530
test feature length: 12540
test feature length: 12550
test feature length: 12560
test feature length: 12570
test feature length: 12580
test feature length: 12590
test feature length: 12600
test feature length: 12610
test feature length: 12620
test feature length: 12630
test feature length: 12640
test feature length: 12650
test feature length: 12660
test feature length: 12670
test feature length: 12680
test feature length: 12690
test feature length: 12700
test feature length: 12710
test feature length: 12720
test feature length: 12730
test feature length: 12740
test feature length: 12750
test feature length: 12760
test feature length: 12770
test feature length: 12780
test feature length: 12790
test feature length: 12800
test feature length: 12810
test feature length: 12820
test feature length: 12830
test feature length: 12840
test feature length: 12850
test feature length: 12860
test feature length: 12870
test feature length: 12880
test feature length: 12890
t

...

test_corrects: tensor(4872, device='cuda:1')
test_corrects: tensor(4881, device='cuda:1')
test_corrects: tensor(4889, device='cuda:1')
test_corrects: tensor(4898, device='cuda:1')
test_corrects: tensor(4907, device='cuda:1')
test_corrects: tensor(4913, device='cuda:1')
test_corrects: tensor(4921, device='cuda:1')
test_corrects: tensor(4931, device='cuda:1')
test_corrects: tensor(4941, device='cuda:1')
test_corrects: tensor(4951, device='cuda:1')
test_corrects: tensor(4961, device='cuda:1')
test_corrects: tensor(4971, device='cuda:1')
test_corrects: tensor(4981, device='cuda:1')
test_corrects: tensor(4991, device='cuda:1')
test_corrects: tensor(5001, device='cuda:1')
test_corrects: tensor(5009, device='cuda:1')
test_corrects: tensor(5015, device='cuda:1')
test_corrects: tensor(5024, device='cuda:1')
test_corrects: tensor(5032, device='cuda:1')
test_corrects: tensor(5042, device='cuda:1')
test_corrects: tensor(5052, device='cuda:1')
test_corrects: tensor(5059, device='cuda:1')
test_corre

test_corrects: tensor(6279, device='cuda:1')
test_corrects: tensor(6289, device='cuda:1')
test_corrects: tensor(6299, device='cuda:1')
test_corrects: tensor(6309, device='cuda:1')
test_corrects: tensor(6319, device='cuda:1')
test_corrects: tensor(6321, device='cuda:1')
test_corrects: tensor(6331, device='cuda:1')
test_corrects: tensor(6341, device='cuda:1')
test_corrects: tensor(6351, device='cuda:1')
test_corrects: tensor(6361, device='cuda:1')
test_corrects: tensor(6371, device='cuda:1')
test_corrects: tensor(6381, device='cuda:1')
test_corrects: tensor(6391, device='cuda:1')
test_corrects: tensor(6401, device='cuda:1')
test_corrects: tensor(6411, device='cuda:1')
test_corrects: tensor(6419, device='cuda:1')
test_corrects: tensor(6427, device='cuda:1')
test_corrects: tensor(6437, device='cuda:1')
test_corrects: tensor(6445, device='cuda:1')
test_corrects: tensor(6453, device='cuda:1')
test_corrects: tensor(6458, device='cuda:1')
test_corrects: tensor(6467, device='cuda:1')
test_corre

test_corrects: tensor(7756, device='cuda:1')
test_corrects: tensor(7758, device='cuda:1')
test_corrects: tensor(7759, device='cuda:1')
test_corrects: tensor(7769, device='cuda:1')
test_corrects: tensor(7779, device='cuda:1')
test_corrects: tensor(7789, device='cuda:1')
test_corrects: tensor(7799, device='cuda:1')
test_corrects: tensor(7808, device='cuda:1')
test_corrects: tensor(7817, device='cuda:1')
test_corrects: tensor(7825, device='cuda:1')
test_corrects: tensor(7835, device='cuda:1')
test_corrects: tensor(7845, device='cuda:1')
test_corrects: tensor(7855, device='cuda:1')
test_corrects: tensor(7865, device='cuda:1')
test_corrects: tensor(7872, device='cuda:1')
test_corrects: tensor(7877, device='cuda:1')
test_corrects: tensor(7885, device='cuda:1')
test_corrects: tensor(7890, device='cuda:1')
test_corrects: tensor(7900, device='cuda:1')
test_corrects: tensor(7900, device='cuda:1')
test_corrects: tensor(7900, device='cuda:1')
test_corrects: tensor(7901, device='cuda:1')
test_corre

test_corrects: tensor(8940, device='cuda:1')
test_corrects: tensor(8949, device='cuda:1')
test_corrects: tensor(8959, device='cuda:1')
test_corrects: tensor(8968, device='cuda:1')
test_corrects: tensor(8978, device='cuda:1')
test_corrects: tensor(8988, device='cuda:1')
test_corrects: tensor(8989, device='cuda:1')
test_corrects: tensor(8992, device='cuda:1')
test_corrects: tensor(8995, device='cuda:1')
test_corrects: tensor(8999, device='cuda:1')
test_corrects: tensor(9002, device='cuda:1')
test_corrects: tensor(9002, device='cuda:1')
test_corrects: tensor(9002, device='cuda:1')
test_corrects: tensor(9008, device='cuda:1')
test_corrects: tensor(9018, device='cuda:1')
test_corrects: tensor(9028, device='cuda:1')
test_corrects: tensor(9038, device='cuda:1')
test_corrects: tensor(9048, device='cuda:1')
test_corrects: tensor(9058, device='cuda:1')
test_corrects: tensor(9066, device='cuda:1')
test_corrects: tensor(9073, device='cuda:1')
test_corrects: tensor(9073, device='cuda:1')
test_corre

test_corrects: tensor(10401, device='cuda:1')
test_corrects: tensor(10405, device='cuda:1')
test_corrects: tensor(10409, device='cuda:1')
test_corrects: tensor(10415, device='cuda:1')
test_corrects: tensor(10418, device='cuda:1')
test_corrects: tensor(10427, device='cuda:1')
test_corrects: tensor(10437, device='cuda:1')
test_corrects: tensor(10447, device='cuda:1')
test_corrects: tensor(10457, device='cuda:1')
test_corrects: tensor(10466, device='cuda:1')
test_corrects: tensor(10476, device='cuda:1')
test_corrects: tensor(10486, device='cuda:1')
test_corrects: tensor(10495, device='cuda:1')
test_corrects: tensor(10505, device='cuda:1')
test_corrects: tensor(10515, device='cuda:1')
test_corrects: tensor(10524, device='cuda:1')
test_corrects: tensor(10534, device='cuda:1')
test_corrects: tensor(10541, device='cuda:1')
test_corrects: tensor(10549, device='cuda:1')
test_corrects: tensor(10559, device='cuda:1')
test_corrects: tensor(10568, device='cuda:1')
test_corrects: tensor(10578, devic

In [18]:
# the first time to test, it is necessary to load feature bank


sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = True # False

MODEL = M5_densenet_lstm_nl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_densenet_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_densenet.pkl"




#########################################
device = "cuda:1"  
model_pure_name= 'NL-densenet' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/tmrnet/models/temp/non-local/densenet_pretrained_only_lr1e-5_L30_2fc_6-25train/latest_model_14.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/densenet169/sl10_lr5e-6_6-25train/latest_model_5.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_15846/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(17, device='cuda:1')
test_corrects: tensor(22, device='cuda:1')
test_corrects: tensor(22, device='cuda:1')
test_corrects: tensor(31, device='cuda:1')
test_corrects: tensor(41, device='cuda:1')
test_corrects: tensor(50, device='cuda:1')
test_corrects: tensor(55, device='cuda:1')
test_corrects: tensor(64, device='cuda:1')
test_corrects: tensor(68, device='cuda:1')
test_corrects: tensor(78, device='cuda:1')
test_corrects: tensor(88, device='cuda:1')
test_corrects: tensor(98, device='cuda:1')
test_corrects: tensor(105, device='cuda:1')
test_corrects: tensor(105, device='cuda:1')
test_corrects: tensor(105, device='cuda:1')
test_corrects: tensor(105, device='cuda:1')
test_corrects: tensor(105, device='cuda:1')
test_corrects: tensor(110, device='cuda:1')
test_corrects: tensor(114, device='cuda:1')
test_corrects: tensor(122, device='cuda:1')
tes

test_corrects: tensor(903, device='cuda:1')
test_corrects: tensor(912, device='cuda:1')
test_corrects: tensor(917, device='cuda:1')
test_corrects: tensor(917, device='cuda:1')
test_corrects: tensor(923, device='cuda:1')
test_corrects: tensor(932, device='cuda:1')
test_corrects: tensor(941, device='cuda:1')
test_corrects: tensor(946, device='cuda:1')
test_corrects: tensor(949, device='cuda:1')
test_corrects: tensor(952, device='cuda:1')
test_corrects: tensor(959, device='cuda:1')
test_corrects: tensor(959, device='cuda:1')
test_corrects: tensor(968, device='cuda:1')
test_corrects: tensor(978, device='cuda:1')
test_corrects: tensor(988, device='cuda:1')
test_corrects: tensor(998, device='cuda:1')
test_corrects: tensor(1008, device='cuda:1')
test_corrects: tensor(1018, device='cuda:1')
test_corrects: tensor(1028, device='cuda:1')
test_corrects: tensor(1038, device='cuda:1')
test_corrects: tensor(1048, device='cuda:1')
test_corrects: tensor(1058, device='cuda:1')
test_corrects: tensor(1062

test_corrects: tensor(2197, device='cuda:1')
test_corrects: tensor(2207, device='cuda:1')
test_corrects: tensor(2217, device='cuda:1')
test_corrects: tensor(2227, device='cuda:1')
test_corrects: tensor(2237, device='cuda:1')
test_corrects: tensor(2245, device='cuda:1')
test_corrects: tensor(2255, device='cuda:1')
test_corrects: tensor(2265, device='cuda:1')
test_corrects: tensor(2275, device='cuda:1')
test_corrects: tensor(2285, device='cuda:1')
test_corrects: tensor(2295, device='cuda:1')
test_corrects: tensor(2304, device='cuda:1')
test_corrects: tensor(2314, device='cuda:1')
test_corrects: tensor(2324, device='cuda:1')
test_corrects: tensor(2334, device='cuda:1')
test_corrects: tensor(2344, device='cuda:1')
test_corrects: tensor(2353, device='cuda:1')
test_corrects: tensor(2359, device='cuda:1')
test_corrects: tensor(2364, device='cuda:1')
test_corrects: tensor(2373, device='cuda:1')
test_corrects: tensor(2383, device='cuda:1')
test_corrects: tensor(2392, device='cuda:1')
test_corre

...

test_corrects: tensor(4892, device='cuda:1')
test_corrects: tensor(4897, device='cuda:1')
test_corrects: tensor(4902, device='cuda:1')
test_corrects: tensor(4907, device='cuda:1')
test_corrects: tensor(4915, device='cuda:1')
test_corrects: tensor(4925, device='cuda:1')
test_corrects: tensor(4931, device='cuda:1')
test_corrects: tensor(4939, device='cuda:1')
test_corrects: tensor(4949, device='cuda:1')
test_corrects: tensor(4959, device='cuda:1')
test_corrects: tensor(4969, device='cuda:1')
test_corrects: tensor(4979, device='cuda:1')
test_corrects: tensor(4989, device='cuda:1')
test_corrects: tensor(4999, device='cuda:1')
test_corrects: tensor(5009, device='cuda:1')
test_corrects: tensor(5019, device='cuda:1')
test_corrects: tensor(5023, device='cuda:1')
test_corrects: tensor(5029, device='cuda:1')
test_corrects: tensor(5038, device='cuda:1')
test_corrects: tensor(5048, device='cuda:1')
test_corrects: tensor(5058, device='cuda:1')
test_corrects: tensor(5062, device='cuda:1')
test_corre

test_corrects: tensor(6303, device='cuda:1')
test_corrects: tensor(6313, device='cuda:1')
test_corrects: tensor(6323, device='cuda:1')
test_corrects: tensor(6333, device='cuda:1')
test_corrects: tensor(6343, device='cuda:1')
test_corrects: tensor(6353, device='cuda:1')
test_corrects: tensor(6362, device='cuda:1')
test_corrects: tensor(6370, device='cuda:1')
test_corrects: tensor(6380, device='cuda:1')
test_corrects: tensor(6390, device='cuda:1')
test_corrects: tensor(6400, device='cuda:1')
test_corrects: tensor(6410, device='cuda:1')
test_corrects: tensor(6420, device='cuda:1')
test_corrects: tensor(6430, device='cuda:1')
test_corrects: tensor(6440, device='cuda:1')
test_corrects: tensor(6450, device='cuda:1')
test_corrects: tensor(6460, device='cuda:1')
test_corrects: tensor(6467, device='cuda:1')
test_corrects: tensor(6477, device='cuda:1')
test_corrects: tensor(6486, device='cuda:1')
test_corrects: tensor(6494, device='cuda:1')
test_corrects: tensor(6500, device='cuda:1')
test_corre

test_corrects: tensor(7800, device='cuda:1')
test_corrects: tensor(7800, device='cuda:1')
test_corrects: tensor(7803, device='cuda:1')
test_corrects: tensor(7803, device='cuda:1')
test_corrects: tensor(7807, device='cuda:1')
test_corrects: tensor(7815, device='cuda:1')
test_corrects: tensor(7824, device='cuda:1')
test_corrects: tensor(7834, device='cuda:1')
test_corrects: tensor(7844, device='cuda:1')
test_corrects: tensor(7849, device='cuda:1')
test_corrects: tensor(7859, device='cuda:1')
test_corrects: tensor(7864, device='cuda:1')
test_corrects: tensor(7864, device='cuda:1')
test_corrects: tensor(7867, device='cuda:1')
test_corrects: tensor(7877, device='cuda:1')
test_corrects: tensor(7887, device='cuda:1')
test_corrects: tensor(7894, device='cuda:1')
test_corrects: tensor(7903, device='cuda:1')
test_corrects: tensor(7912, device='cuda:1')
test_corrects: tensor(7922, device='cuda:1')
test_corrects: tensor(7932, device='cuda:1')
test_corrects: tensor(7941, device='cuda:1')
test_corre

test_corrects: tensor(8949, device='cuda:1')
test_corrects: tensor(8958, device='cuda:1')
test_corrects: tensor(8968, device='cuda:1')
test_corrects: tensor(8978, device='cuda:1')
test_corrects: tensor(8988, device='cuda:1')
test_corrects: tensor(8998, device='cuda:1')
test_corrects: tensor(9008, device='cuda:1')
test_corrects: tensor(9016, device='cuda:1')
test_corrects: tensor(9022, device='cuda:1')
test_corrects: tensor(9029, device='cuda:1')
test_corrects: tensor(9033, device='cuda:1')
test_corrects: tensor(9042, device='cuda:1')
test_corrects: tensor(9052, device='cuda:1')
test_corrects: tensor(9060, device='cuda:1')
test_corrects: tensor(9067, device='cuda:1')
test_corrects: tensor(9075, device='cuda:1')
test_corrects: tensor(9085, device='cuda:1')
test_corrects: tensor(9095, device='cuda:1')
test_corrects: tensor(9105, device='cuda:1')
test_corrects: tensor(9115, device='cuda:1')
test_corrects: tensor(9123, device='cuda:1')
test_corrects: tensor(9129, device='cuda:1')
test_corre

test_corrects: tensor(10443, device='cuda:1')
test_corrects: tensor(10453, device='cuda:1')
test_corrects: tensor(10457, device='cuda:1')
test_corrects: tensor(10461, device='cuda:1')
test_corrects: tensor(10464, device='cuda:1')
test_corrects: tensor(10471, device='cuda:1')
test_corrects: tensor(10481, device='cuda:1')
test_corrects: tensor(10491, device='cuda:1')
test_corrects: tensor(10501, device='cuda:1')
test_corrects: tensor(10511, device='cuda:1')
test_corrects: tensor(10521, device='cuda:1')
test_corrects: tensor(10531, device='cuda:1')
test_corrects: tensor(10537, device='cuda:1')
test_corrects: tensor(10543, device='cuda:1')
test_corrects: tensor(10553, device='cuda:1')
test_corrects: tensor(10563, device='cuda:1')
test_corrects: tensor(10573, device='cuda:1')
test_corrects: tensor(10583, device='cuda:1')
test_corrects: tensor(10593, device='cuda:1')
test_corrects: tensor(10603, device='cuda:1')
test_corrects: tensor(10613, device='cuda:1')
test_corrects: tensor(10623, devic

In [16]:
# the first time to test, it is necessary to load feature bank


sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = True # False

MODEL = M8_densenet_lstm_nl_tvl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_densenet_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_densenet.pkl"




#########################################
device = "cuda:1"  
model_pure_name= 'NL-TVL-densenet' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/tmrnet/models/temp/non-local/densenet_pretrained_lr1e-5_L30_multiconv_2fc_6-25train/latest_model_1.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/densenet169/sl10_lr5e-6_6-25train/latest_model_5.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_10186/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(17, device='cuda:1')
test_corrects: tensor(20, device='cuda:1')
test_corrects: tensor(20, device='cuda:1')
test_corrects: tensor(28, device='cuda:1')
test_corrects: tensor(38, device='cuda:1')
test_corrects: tensor(47, device='cuda:1')
test_corrects: tensor(54, device='cuda:1')
test_corrects: tensor(64, device='cuda:1')
test_corrects: tensor(69, device='cuda:1')
test_corrects: tensor(79, device='cuda:1')
test_corrects: tensor(89, device='cuda:1')
test_corrects: tensor(99, device='cuda:1')
test_corrects: tensor(107, device='cuda:1')
test_corrects: tensor(107, device='cuda:1')
test_corrects: tensor(107, device='cuda:1')
test_corrects: tensor(107, device='cuda:1')
test_corrects: tensor(117, device='cuda:1')
test_corrects: tensor(126, device='cuda:1')
test_corrects: tensor(133, device='cuda:1')
test_corrects: tensor(133, device='cuda:1')
tes

test_corrects: tensor(761, device='cuda:1')
test_corrects: tensor(765, device='cuda:1')
test_corrects: tensor(772, device='cuda:1')
test_corrects: tensor(774, device='cuda:1')
test_corrects: tensor(774, device='cuda:1')
test_corrects: tensor(780, device='cuda:1')
test_corrects: tensor(790, device='cuda:1')
test_corrects: tensor(799, device='cuda:1')
test_corrects: tensor(806, device='cuda:1')
test_corrects: tensor(810, device='cuda:1')
test_corrects: tensor(812, device='cuda:1')
test_corrects: tensor(819, device='cuda:1')
test_corrects: tensor(819, device='cuda:1')
test_corrects: tensor(825, device='cuda:1')
test_corrects: tensor(832, device='cuda:1')
test_corrects: tensor(842, device='cuda:1')
test_corrects: tensor(847, device='cuda:1')
test_corrects: tensor(857, device='cuda:1')
test_corrects: tensor(867, device='cuda:1')
test_corrects: tensor(877, device='cuda:1')
test_corrects: tensor(887, device='cuda:1')
test_corrects: tensor(897, device='cuda:1')
test_corrects: tensor(907, devic

test_corrects: tensor(2047, device='cuda:1')
test_corrects: tensor(2057, device='cuda:1')
test_corrects: tensor(2067, device='cuda:1')
test_corrects: tensor(2077, device='cuda:1')
test_corrects: tensor(2087, device='cuda:1')
test_corrects: tensor(2097, device='cuda:1')
test_corrects: tensor(2105, device='cuda:1')
test_corrects: tensor(2115, device='cuda:1')
test_corrects: tensor(2125, device='cuda:1')
test_corrects: tensor(2135, device='cuda:1')
test_corrects: tensor(2145, device='cuda:1')
test_corrects: tensor(2154, device='cuda:1')
test_corrects: tensor(2161, device='cuda:1')
test_corrects: tensor(2171, device='cuda:1')
test_corrects: tensor(2181, device='cuda:1')
test_corrects: tensor(2191, device='cuda:1')
test_corrects: tensor(2201, device='cuda:1')
test_corrects: tensor(2210, device='cuda:1')
test_corrects: tensor(2211, device='cuda:1')
test_corrects: tensor(2217, device='cuda:1')
test_corrects: tensor(2226, device='cuda:1')
test_corrects: tensor(2236, device='cuda:1')
test_corre

...

test_corrects: tensor(4670, device='cuda:1')
test_corrects: tensor(4680, device='cuda:1')
test_corrects: tensor(4685, device='cuda:1')
test_corrects: tensor(4685, device='cuda:1')
test_corrects: tensor(4691, device='cuda:1')
test_corrects: tensor(4700, device='cuda:1')
test_corrects: tensor(4709, device='cuda:1')
test_corrects: tensor(4715, device='cuda:1')
test_corrects: tensor(4725, device='cuda:1')
test_corrects: tensor(4735, device='cuda:1')
test_corrects: tensor(4745, device='cuda:1')
test_corrects: tensor(4755, device='cuda:1')
test_corrects: tensor(4765, device='cuda:1')
test_corrects: tensor(4775, device='cuda:1')
test_corrects: tensor(4785, device='cuda:1')
test_corrects: tensor(4795, device='cuda:1')
test_corrects: tensor(4805, device='cuda:1')
test_corrects: tensor(4810, device='cuda:1')
test_corrects: tensor(4816, device='cuda:1')
test_corrects: tensor(4824, device='cuda:1')
test_corrects: tensor(4834, device='cuda:1')
test_corrects: tensor(4844, device='cuda:1')
test_corre

test_corrects: tensor(6092, device='cuda:1')
test_corrects: tensor(6102, device='cuda:1')
test_corrects: tensor(6112, device='cuda:1')
test_corrects: tensor(6122, device='cuda:1')
test_corrects: tensor(6132, device='cuda:1')
test_corrects: tensor(6142, device='cuda:1')
test_corrects: tensor(6152, device='cuda:1')
test_corrects: tensor(6161, device='cuda:1')
test_corrects: tensor(6169, device='cuda:1')
test_corrects: tensor(6179, device='cuda:1')
test_corrects: tensor(6189, device='cuda:1')
test_corrects: tensor(6199, device='cuda:1')
test_corrects: tensor(6209, device='cuda:1')
test_corrects: tensor(6219, device='cuda:1')
test_corrects: tensor(6229, device='cuda:1')
test_corrects: tensor(6239, device='cuda:1')
test_corrects: tensor(6249, device='cuda:1')
test_corrects: tensor(6259, device='cuda:1')
test_corrects: tensor(6269, device='cuda:1')
test_corrects: tensor(6279, device='cuda:1')
test_corrects: tensor(6289, device='cuda:1')
test_corrects: tensor(6297, device='cuda:1')
test_corre

test_corrects: tensor(7590, device='cuda:1')
test_corrects: tensor(7592, device='cuda:1')
test_corrects: tensor(7592, device='cuda:1')
test_corrects: tensor(7593, device='cuda:1')
test_corrects: tensor(7593, device='cuda:1')
test_corrects: tensor(7598, device='cuda:1')
test_corrects: tensor(7608, device='cuda:1')
test_corrects: tensor(7618, device='cuda:1')
test_corrects: tensor(7628, device='cuda:1')
test_corrects: tensor(7638, device='cuda:1')
test_corrects: tensor(7646, device='cuda:1')
test_corrects: tensor(7655, device='cuda:1')
test_corrects: tensor(7663, device='cuda:1')
test_corrects: tensor(7663, device='cuda:1')
test_corrects: tensor(7666, device='cuda:1')
test_corrects: tensor(7676, device='cuda:1')
test_corrects: tensor(7686, device='cuda:1')
test_corrects: tensor(7690, device='cuda:1')
test_corrects: tensor(7700, device='cuda:1')
test_corrects: tensor(7704, device='cuda:1')
test_corrects: tensor(7714, device='cuda:1')
test_corrects: tensor(7724, device='cuda:1')
test_corre

test_corrects: tensor(8726, device='cuda:1')
test_corrects: tensor(8736, device='cuda:1')
test_corrects: tensor(8742, device='cuda:1')
test_corrects: tensor(8750, device='cuda:1')
test_corrects: tensor(8760, device='cuda:1')
test_corrects: tensor(8770, device='cuda:1')
test_corrects: tensor(8780, device='cuda:1')
test_corrects: tensor(8790, device='cuda:1')
test_corrects: tensor(8797, device='cuda:1')
test_corrects: tensor(8803, device='cuda:1')
test_corrects: tensor(8806, device='cuda:1')
test_corrects: tensor(8810, device='cuda:1')
test_corrects: tensor(8815, device='cuda:1')
test_corrects: tensor(8821, device='cuda:1')
test_corrects: tensor(8823, device='cuda:1')
test_corrects: tensor(8827, device='cuda:1')
test_corrects: tensor(8835, device='cuda:1')
test_corrects: tensor(8845, device='cuda:1')
test_corrects: tensor(8855, device='cuda:1')
test_corrects: tensor(8865, device='cuda:1')
test_corrects: tensor(8875, device='cuda:1')
test_corrects: tensor(8883, device='cuda:1')
test_corre

test_corrects: tensor(10168, device='cuda:1')
test_corrects: tensor(10175, device='cuda:1')
test_corrects: tensor(10184, device='cuda:1')
test_corrects: tensor(10188, device='cuda:1')
test_corrects: tensor(10188, device='cuda:1')
test_corrects: tensor(10188, device='cuda:1')
test_corrects: tensor(10189, device='cuda:1')
test_corrects: tensor(10199, device='cuda:1')
test_corrects: tensor(10209, device='cuda:1')
test_corrects: tensor(10219, device='cuda:1')
test_corrects: tensor(10229, device='cuda:1')
test_corrects: tensor(10239, device='cuda:1')
test_corrects: tensor(10249, device='cuda:1')
test_corrects: tensor(10253, device='cuda:1')
test_corrects: tensor(10260, device='cuda:1')
test_corrects: tensor(10270, device='cuda:1')
test_corrects: tensor(10280, device='cuda:1')
test_corrects: tensor(10290, device='cuda:1')
test_corrects: tensor(10300, device='cuda:1')
test_corrects: tensor(10310, device='cuda:1')
test_corrects: tensor(10320, device='cuda:1')
test_corrects: tensor(10330, devic

In [21]:
# the first time to test, it is necessary to load feature bank

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = True # False

MODEL = M8_densenet_lstm_nl_tvl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_densenet_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_densenet.pkl"




#########################################
device = "cuda:1"  
model_pure_name= 'NL-TVL-densenet' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/tmrnet/models/temp/non-local/densenet_pretrained_lr1e-5_L30_multiconv_2fc_6-25train/latest_model_3.pth"

# lfb model path
LFB_path = "/media/yilin/tmrnet/models/temp/densenet169/sl10_lr5e-6_6-25train/latest_model_5.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_15846/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(9, device='cuda:1')
test_corrects: tensor(14, device='cuda:1')
test_corrects: tensor(14, device='cuda:1')
test_corrects: tensor(17, device='cuda:1')
test_corrects: tensor(20, device='cuda:1')
test_corrects: tensor(20, device='cuda:1')
test_corrects: tensor(25, device='cuda:1')
test_corrects: tensor(35, device='cuda:1')
test_corrects: tensor(45, device='cuda:1')
test_corrects: tensor(52, device='cuda:1')
test_corrects: tensor(61, device='cuda:1')
test_corrects: tensor(62, device='cuda:1')
test_corrects: tensor(70, device='cuda:1')
test_corrects: tensor(80, device='cuda:1')
test_corrects: tensor(90, device='cuda:1')
test_corrects: tensor(100, device='cuda:1')
test_corrects: tensor(100, device='cuda:1')
test_corrects: tensor(100, device='cuda:1')
test_corrects: tensor(100, device='cuda:1')
test_corrects: tensor(101, device='cuda:1')
test_corrects: tensor(110, device='cuda:1')
test_corrects: tensor(120, device='cuda:1')
test_corrects: tensor(125, device='cuda:1')
test

test_corrects: tensor(679, device='cuda:1')
test_corrects: tensor(681, device='cuda:1')
test_corrects: tensor(682, device='cuda:1')
test_corrects: tensor(682, device='cuda:1')
test_corrects: tensor(688, device='cuda:1')
test_corrects: tensor(697, device='cuda:1')
test_corrects: tensor(704, device='cuda:1')
test_corrects: tensor(714, device='cuda:1')
test_corrects: tensor(719, device='cuda:1')
test_corrects: tensor(721, device='cuda:1')
test_corrects: tensor(726, device='cuda:1')
test_corrects: tensor(726, device='cuda:1')
test_corrects: tensor(734, device='cuda:1')
test_corrects: tensor(743, device='cuda:1')
test_corrects: tensor(753, device='cuda:1')
test_corrects: tensor(760, device='cuda:1')
test_corrects: tensor(770, device='cuda:1')
test_corrects: tensor(777, device='cuda:1')
test_corrects: tensor(784, device='cuda:1')
test_corrects: tensor(794, device='cuda:1')
test_corrects: tensor(804, device='cuda:1')
test_corrects: tensor(814, device='cuda:1')
test_corrects: tensor(818, devic

test_corrects: tensor(1925, device='cuda:1')
test_corrects: tensor(1935, device='cuda:1')
test_corrects: tensor(1945, device='cuda:1')
test_corrects: tensor(1955, device='cuda:1')
test_corrects: tensor(1963, device='cuda:1')
test_corrects: tensor(1973, device='cuda:1')
test_corrects: tensor(1983, device='cuda:1')
test_corrects: tensor(1993, device='cuda:1')
test_corrects: tensor(2003, device='cuda:1')
test_corrects: tensor(2013, device='cuda:1')
test_corrects: tensor(2020, device='cuda:1')
test_corrects: tensor(2030, device='cuda:1')
test_corrects: tensor(2040, device='cuda:1')
test_corrects: tensor(2050, device='cuda:1')
test_corrects: tensor(2060, device='cuda:1')
test_corrects: tensor(2069, device='cuda:1')
test_corrects: tensor(2079, device='cuda:1')
test_corrects: tensor(2085, device='cuda:1')
test_corrects: tensor(2094, device='cuda:1')
test_corrects: tensor(2104, device='cuda:1')
test_corrects: tensor(2113, device='cuda:1')
test_corrects: tensor(2123, device='cuda:1')
test_corre

...

test_corrects: tensor(4573, device='cuda:1')
test_corrects: tensor(4574, device='cuda:1')
test_corrects: tensor(4578, device='cuda:1')
test_corrects: tensor(4585, device='cuda:1')
test_corrects: tensor(4595, device='cuda:1')
test_corrects: tensor(4600, device='cuda:1')
test_corrects: tensor(4609, device='cuda:1')
test_corrects: tensor(4619, device='cuda:1')
test_corrects: tensor(4629, device='cuda:1')
test_corrects: tensor(4639, device='cuda:1')
test_corrects: tensor(4649, device='cuda:1')
test_corrects: tensor(4659, device='cuda:1')
test_corrects: tensor(4669, device='cuda:1')
test_corrects: tensor(4679, device='cuda:1')
test_corrects: tensor(4689, device='cuda:1')
test_corrects: tensor(4694, device='cuda:1')
test_corrects: tensor(4700, device='cuda:1')
test_corrects: tensor(4709, device='cuda:1')
test_corrects: tensor(4719, device='cuda:1')
test_corrects: tensor(4729, device='cuda:1')
test_corrects: tensor(4733, device='cuda:1')
test_corrects: tensor(4733, device='cuda:1')
test_corre

test_corrects: tensor(6004, device='cuda:1')
test_corrects: tensor(6014, device='cuda:1')
test_corrects: tensor(6024, device='cuda:1')
test_corrects: tensor(6034, device='cuda:1')
test_corrects: tensor(6044, device='cuda:1')
test_corrects: tensor(6044, device='cuda:1')
test_corrects: tensor(6052, device='cuda:1')
test_corrects: tensor(6062, device='cuda:1')
test_corrects: tensor(6072, device='cuda:1')
test_corrects: tensor(6082, device='cuda:1')
test_corrects: tensor(6092, device='cuda:1')
test_corrects: tensor(6102, device='cuda:1')
test_corrects: tensor(6112, device='cuda:1')
test_corrects: tensor(6122, device='cuda:1')
test_corrects: tensor(6132, device='cuda:1')
test_corrects: tensor(6142, device='cuda:1')
test_corrects: tensor(6151, device='cuda:1')
test_corrects: tensor(6161, device='cuda:1')
test_corrects: tensor(6170, device='cuda:1')
test_corrects: tensor(6178, device='cuda:1')
test_corrects: tensor(6180, device='cuda:1')
test_corrects: tensor(6189, device='cuda:1')
test_corre

test_corrects: tensor(7503, device='cuda:1')
test_corrects: tensor(7504, device='cuda:1')
test_corrects: tensor(7504, device='cuda:1')
test_corrects: tensor(7508, device='cuda:1')
test_corrects: tensor(7518, device='cuda:1')
test_corrects: tensor(7528, device='cuda:1')
test_corrects: tensor(7538, device='cuda:1')
test_corrects: tensor(7548, device='cuda:1')
test_corrects: tensor(7552, device='cuda:1')
test_corrects: tensor(7561, device='cuda:1')
test_corrects: tensor(7569, device='cuda:1')
test_corrects: tensor(7569, device='cuda:1')
test_corrects: tensor(7572, device='cuda:1')
test_corrects: tensor(7582, device='cuda:1')
test_corrects: tensor(7592, device='cuda:1')
test_corrects: tensor(7600, device='cuda:1')
test_corrects: tensor(7610, device='cuda:1')
test_corrects: tensor(7612, device='cuda:1')
test_corrects: tensor(7622, device='cuda:1')
test_corrects: tensor(7628, device='cuda:1')
test_corrects: tensor(7637, device='cuda:1')
test_corrects: tensor(7640, device='cuda:1')
test_corre

test_corrects: tensor(8639, device='cuda:1')
test_corrects: tensor(8648, device='cuda:1')
test_corrects: tensor(8657, device='cuda:1')
test_corrects: tensor(8667, device='cuda:1')
test_corrects: tensor(8677, device='cuda:1')
test_corrects: tensor(8687, device='cuda:1')
test_corrects: tensor(8697, device='cuda:1')
test_corrects: tensor(8705, device='cuda:1')
test_corrects: tensor(8711, device='cuda:1')
test_corrects: tensor(8714, device='cuda:1')
test_corrects: tensor(8720, device='cuda:1')
test_corrects: tensor(8724, device='cuda:1')
test_corrects: tensor(8724, device='cuda:1')
test_corrects: tensor(8725, device='cuda:1')
test_corrects: tensor(8733, device='cuda:1')
test_corrects: tensor(8742, device='cuda:1')
test_corrects: tensor(8752, device='cuda:1')
test_corrects: tensor(8762, device='cuda:1')
test_corrects: tensor(8772, device='cuda:1')
test_corrects: tensor(8782, device='cuda:1')
test_corrects: tensor(8790, device='cuda:1')
test_corrects: tensor(8798, device='cuda:1')
test_corre

test_corrects: tensor(10107, device='cuda:1')
test_corrects: tensor(10115, device='cuda:1')
test_corrects: tensor(10119, device='cuda:1')
test_corrects: tensor(10120, device='cuda:1')
test_corrects: tensor(10124, device='cuda:1')
test_corrects: tensor(10124, device='cuda:1')
test_corrects: tensor(10134, device='cuda:1')
test_corrects: tensor(10144, device='cuda:1')
test_corrects: tensor(10154, device='cuda:1')
test_corrects: tensor(10164, device='cuda:1')
test_corrects: tensor(10174, device='cuda:1')
test_corrects: tensor(10184, device='cuda:1')
test_corrects: tensor(10188, device='cuda:1')
test_corrects: tensor(10193, device='cuda:1')
test_corrects: tensor(10203, device='cuda:1')
test_corrects: tensor(10213, device='cuda:1')
test_corrects: tensor(10222, device='cuda:1')
test_corrects: tensor(10232, device='cuda:1')
test_corrects: tensor(10241, device='cuda:1')
test_corrects: tensor(10251, device='cuda:1')
test_corrects: tensor(10261, device='cuda:1')
test_corrects: tensor(10271, devic